In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#IMPORTS
import numpy as np
import os
import matplotlib.pyplot as plt
import os
from owi_data_2_pandas.io import API
import datetime
from pytz import utc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import dotenv
dotenv.load_dotenv()

from dw_config.location import Turbine
from dw_database.utils import DynawindFrame

In [3]:
def get_big_data(start, end, owi_api, location, metrics, days=15):
    new_start = start
    all_data = pd.DataFrame()
    middle = start + datetime.timedelta(days=days)
    while end > middle:
        data_ = owi_api.query((new_start, middle), location=location, metrics = metrics)
        new_start = middle
        middle = new_start + datetime.timedelta(days=days)
        all_data = pd.concat([all_data, data_])
    if end < middle:
        data_ = owi_api.query((new_start, end), location=location, metrics = metrics)
        all_data = pd.concat([all_data, data_])
    duplicated_indices = all_data.index.duplicated(keep='first')
    all_data_unique = all_data[~duplicated_indices]
    return all_data_unique

def rename_mpe_cols(data: pd.DataFrame, replcement:str = 'mode_'):
    for col in data.columns:
        if 'mpe_' in col:
            data.rename(columns={col: col.replace('mpe_', replcement)}, inplace=True)
    return data

In [4]:
from owi_data_2_pandas.io import API

# Get the turbine data through the API
user = os.getenv('API_USERNAME') 
password = os.getenv('API_PASSWORD')
root = r"http://10.0.0.138/api/v1/"


owi_api = API(api_root=root, username=user, password=password)

In [5]:
# GET THE DATA
package_folder = os.path.dirname(os.path.dirname(os.getcwd()))
data_folder = os.path.join(package_folder, 'data')
data_folder

'c:\\Users\\Max\\Documents\\OWI_LAB\\Code\\DCE_EOVnorm\\dce_eov_normalisation\\data'

In [6]:
start = datetime.datetime(2022, 10, 1, tzinfo=utc)
end = datetime.datetime(2023, 12, 1, tzinfo=utc)

new_data_folder = os.path.join(data_folder, 'raw', 'nw2')
new_caseID_folder = os.path.join(data_folder, 'interim', 'nw2', 'labeled')

for location in ['nw2c02']: #owi_api.groups[owi_api.groups.index == '4']['locations'][0]:
    print(location)
    metrics = owi_api.metrics(locations=location)
    data = get_big_data(start, end, owi_api, location, metrics.metric, days=10)
    
    new_data_path = os.path.join(new_data_folder, location+'.parquet')
    data.to_parquet(new_data_path)

    data_renamed = rename_mpe_cols(data, replcement='mode_').drop('location', axis=1)
    turbine = Turbine(location.upper())
    dwf_ = DynawindFrame(df=data_renamed, location=turbine)
    case_stats = dwf_.case_stats()
    dwf_[['caseID']].to_csv(os.path.join(new_caseID_folder, 'caseIDs_Alexandra', location+'_caseIDs.csv'))
    

nw2c02


In [20]:
dwf_[['caseID']].to_csv(os.path.join(data_folder, 'interim', 'nw2','labeled', location+'_case.csv'))

timestamp
2022-10-01 00:00:00+00:00                                                 None
2022-10-01 00:10:00+00:00         Rated power: Turbine at rated power (9525kW)
2022-10-01 00:20:00+00:00         Rated power: Turbine at rated power (9525kW)
2022-10-01 00:30:00+00:00         Rated power: Turbine at rated power (9525kW)
2022-10-01 00:40:00+00:00         Rated power: Turbine at rated power (9525kW)
                                                   ...                        
2023-11-30 23:20:00+00:00    Curtailed (All): Catchall for data points that...
2023-11-30 23:30:00+00:00    Curtailed (All): Catchall for data points that...
2023-11-30 23:40:00+00:00            rpm<Max: Turbine speeding up to rated RPM
2023-11-30 23:50:00+00:00                  rpm6.6: Turbine rotating at 6.6 RPM
2023-12-01 00:00:00+00:00            rpm<Max: Turbine speeding up to rated RPM
Name: caseID, Length: 61292, dtype: object